In [1]:
import random

# Example Cebuano conversation templates
conversations = [
    ("Kumusta ka?", "Maayo ra ko, ikaw?"),
    ("Asa ka padulong?", "Pauli na ko sa balay."),
    ("Unsa imong pangalan?", "Akong pangalan kay Juan."),
    ("Ganahan ka ug kape?", "Oo, ganahan kaayo ko."),
    ("Nindot ba ang panahon karon?", "Oo, hayag ang adlaw."),
    ("Asa ta magkaon?", "Sa luyo nga karenderya, lami didto."),
    ("Unsa imong buhaton karon?", "Magtrabaho ra ko."),
    ("Nakapalit na ka ug bugas?", "Wala pa, mangita ko ug tindahan."),
    ("Pila imong edad?", "Bente tres ako."),
    ("Kanus-a ka miabot?", "Bag-o lang, mga alas dos."),
    ("Unsa imong hilig?", "Ganahan ko ug basketball ug music."),
    ("Nagtrabaho ka ba?", "Oo, sa opisina ko nagtrabaho."),
    ("Asa ta magkita ugma?", "Sa plaza, alas nuwebe sa buntag."),
    ("Ganahan ka moadto sa dagat?", "Oo, excited na kaayo ko!"),
    ("Unsa imong gi-order?", "Chicken ug rice ra."),
    ("Nakakita ka sa akong libro?", "Wala pa, asa man nimo gibutang?"),
    ("Nalimtan nimo ang susi?", "Wala, naa ra sa akong bulsa."),
    ("Kinsa imong kuyog?", "Akong igsuon ra."),
    ("Dugay na ka sa Cebu?", "Oo, lima na ka tuig."),
    ("Unsa imong plano ugma?", "Maglaag ra siguro sa mall.")
]

# Generate synthetic dataset with conversation pairs
dataset = []
# for i in range(20):
#     question, answer = random.choice(conversations)
#     dataset.append({"conversation_id": i + 1, "question": question, "answer": answer})

i=1
for conversation in conversations:
    question, answer = conversation[0], conversation[1]
    dataset.append({"conversation_id":i,"question":question, "answer":answer})
    i+=1

# Save the dataset to JSON
import json

with open("cebuano_conversations.json", "w") as f:
    json.dump(dataset, f, indent=4, ensure_ascii=False)

print("Generated Cebuano conversation dataset!")


Generated Cebuano conversation dataset!


In [2]:
from datasets import load_dataset

# Load the custom dataset
dataset = load_dataset("json", data_files="cebuano_conversations.json")

# Tokenize the dataset
from transformers import AutoTokenizer

# Use a pre-trained tokenizer (you can use GPT-2 or a multilingual model like mBERT)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def tokenize_function(examples):
    combined_texts = [q + " " + a for q, a in zip(examples["question"], examples["answer"])]
    return tokenizer(combined_texts, truncation=True, max_length=128, padding="max_length")




python(42467) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Generating train split: 0 examples [00:00, ? examples/s]

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from transformers import GPT2LMHeadModel

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Resize the embedding layer to match the vocabulary size (if necessary)
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [4]:
# Add padding token if not already present
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize the model's embeddings to include the new token

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["conversation_id", "question", "answer"])
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [5]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./cebuano_transformer",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=5,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

# Define the Trainer
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],  # In a real scenario, use a separate validation set
    data_collator=data_collator,
)

# Train the model
trainer.train()


/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,9.386000
2,No log,5.653410
3,No log,4.849442
4,No log,4.430984
5,No log,4.283127


TrainOutput(global_step=50, training_loss=11.43436767578125, metrics={'train_runtime': 21.7509, 'train_samples_per_second': 4.598, 'train_steps_per_second': 2.299, 'total_flos': 6532300800000.0, 'train_loss': 11.43436767578125, 'epoch': 5.0})

In [6]:
# Save the model and tokenizer
model.save_pretrained("./cebuano_transformer")
tokenizer.save_pretrained("./cebuano_transformer")

print("Model and tokenizer saved to './cebuano_transformer'")


Model and tokenizer saved to './cebuano_transformer'


In [9]:
from transformers import GPT2LMHeadModel, AutoTokenizer, pipeline

# Load the saved model and tokenizer
loaded_model = GPT2LMHeadModel.from_pretrained("./cebuano_transformer")
loaded_tokenizer = AutoTokenizer.from_pretrained("./cebuano_transformer")

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [10]:
tokenizer = AutoTokenizer.from_pretrained("./cebuano_transformer")

# Create text generation pipeline
generator = pipeline("text-generation", model=loaded_model, tokenizer=loaded_tokenizer)

# Generate text
prompt = "Kumusta ka?"
output = generator(prompt, max_length=50, num_return_sequences=1)

print("Generated Cebuano Response:")
print(output[0]["generated_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Cebuano Response:
Kumusta ka? Walaat sa. na kulapag ka kot. Karabay kilala sa. kapapawala na kumang na kalakad. Hindi saan. Walaat


In [19]:

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.197862
2,No log,0.165616
3,No log,0.162566
4,No log,0.163571
5,No log,0.157410


TrainOutput(global_step=50, training_loss=0.32314529418945315, metrics={'train_runtime': 20.5958, 'train_samples_per_second': 4.855, 'train_steps_per_second': 2.428, 'total_flos': 6532300800000.0, 'train_loss': 0.32314529418945315, 'epoch': 5.0})

In [20]:
# Load the saved model and tokenizer
loaded_model2 = GPT2LMHeadModel.from_pretrained("./cebuano_transformer")
loaded_tokenizer2 = AutoTokenizer.from_pretrained("./cebuano_transformer")

print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [21]:
tokenizer = AutoTokenizer.from_pretrained("./cebuano_transformer")

# Create text generation pipeline
generator = pipeline("text-generation", model=loaded_model, tokenizer=loaded_tokenizer)

# Generate text
prompt = "Kumusta ka?"
print("Prompt: " +prompt+"\n")
output = generator(prompt, max_length=50, num_return_sequences=1)

print("Generated Cebuano Response:")
print(output[0]["generated_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prompt: Kumusta ka?

Generated Cebuano Response:
Kumusta ka?
ga?
ayo ka sa mga. O ka sa gag na.

. nga, lindan sa kanig!
ahan ko sa ba ko.
ang


In [ ]:

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./cebuano_transformer3")
tokenizer.save_pretrained("./cebuano_transformer3")

tokenizer = AutoTokenizer.from_pretrained("./cebuano_transformer3")

# Create text generation pipeline
generator = pipeline("text-generation", model=loaded_model, tokenizer=loaded_tokenizer)

# Generate text
prompt = "Kumusta ka?"
print("Prompt: " +prompt+"\n")
output = generator(prompt, max_length=50, num_return_sequences=1)

print("Generated Cebuano Response:")
print(output[0]["generated_text"])